## Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

### Downloading the data

In [2]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareclimate_data/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]

In [4]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 5.31 s, sys: 5.23 s, total: 10.5 s
Wall time: 2min 6s


### Extract contents of zipped file

In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(f'{output_directory}/data')

CPU times: user 14.4 s, sys: 925 ms, total: 15.3 s
Wall time: 15.6 s


### Remove unused file

In [6]:
unused_file = os.path.join(
    output_directory,
    "data/observed_daily_rainfall_SYD.csv")
if os.path.exists(unused_file):
    os.remove(unused_file)

### Combining data CSVs

In [7]:
%%time
files = glob.glob('figshareclimate_data/data/*.csv')
df = pd.concat(
    (pd.read_csv(file, index_col=0, parse_dates=['time'])
     .assign(model=re.findall(r'[^\/]+(?=_daily_rainfall_NSW\.)', file)[0])
     for file in files)
)
df.to_csv("figshareclimate_data/combined_data.csv")

CPU times: user 6min 47s, sys: 11.5 s, total: 6min 59s
Wall time: 7min 12s


In [8]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


### EDA